In [2]:
import json
import sys
from pycocotools.coco import COCO
import numpy as np
import matplotlib.pyplot as plt
import os
import shutil

In [50]:
main_folder = '13zebra'
exps = os.listdir(main_folder)
outdir = os.path.join('.','ap36k_zebra')
outdir_images = os.path.join(outdir,'images')
outdir_labels = os.path.join(outdir,'labels')
outdir_labels_coco = os.path.join(outdir,'labels_coco')
for od in [outdir, outdir_images, outdir_labels, outdir_labels_coco]:
    if not os.path.exists(od):
        os.makedirs(od)
        
for exp in exps:
    files = os.listdir(os.path.join(main_folder,exp))
    for file in files:
        if file[-5:] == '.json':
            fname = file[:-5]
            out_fname = f'{exp}_{fname}'
            shutil.copy(os.path.join(main_folder,exp,f"{fname}.jpg"),os.path.join(outdir_images,out_fname+".jpg"))
            with open(os.path.join(main_folder,exp,f"{fname}.json"),"r") as f:
                annos = json.load(f)
                imwidth = annos['imageWidth']
                imheight = annos['imageHeight']
                
                with open(os.path.join(outdir_labels,out_fname+".txt"),'w') as of, open(os.path.join(outdir_labels_coco,out_fname+".txt"),'w') as of_coco:
                    for shape in annos['shapes']:
                        if shape['shape_type'] == 'rectangle': # is bbox
                            c = 0
                            x_center = (shape['points'][0][0] + shape['points'][1][0])/(2*imwidth)
                            y_center = (shape['points'][0][1] + shape['points'][1][1])/(2*imheight)
                            lx = min(shape['points'][0][0], shape['points'][1][0])
                            rx = max(shape['points'][0][0], shape['points'][1][0])
                            uy = min(shape['points'][0][1], shape['points'][1][1])
                            dy = max(shape['points'][0][1], shape['points'][1][1])
                            width = (rx-lx)/(imwidth)
                            height = (dy-uy)/(imheight)
                            of.write(f"{c} {x_center} {y_center} {width} {height}\n")
                            c = 22
                            of_coco.write(f"{c} {x_center} {y_center} {width} {height}\n")


# create train and val jsons and copy the images to the target folders

In [4]:
myl = ['video2','video5','video13','video19']

In [138]:
def map_keypoints(k):
    tmp = [0]*81
    tmp[0:3] = k[39:42] # left back paw
    tmp[3:6] = k[36:39] # left back knee
    tmp[6:9] = k[33:36] # left back thigh
    
    tmp[9:12]  = k[48:51] # r back paw
    tmp[12:15] = k[45:48] # r back knee
    tmp[15:18] = k[42:45] # r back thigh
    
    tmp[18:21] = k[30:33] # r f p
    tmp[21:24] = k[27:30] # r f k
    tmp[24:27] = k[24:27] # r f t
    
    tmp[27:30] = k[21:24] # l f p
    tmp[30:33] = k[18:21] # l f k
    tmp[33:36] = k[15:18] # l f t
    
    #tmp[36:39]
    #tmp[39:42] = k[12:15] # root of tail
    #tmp[42:45, 45:48, 48:51, 51:54]
    
    tmp[54:57] = k[3:6] # right eye
    tmp[57:60] = k[0:3] # left eye
    tmp[60:63] = k[6:9] # nose
    # 63-66, 66-69, 69-72, 72-75, 75-78, 78-81
    return tmp

In [65]:
f = json.load(open("/media/ebonetto/WindowsData/pose_zebras_sw/ViTPose/data/apt36k_annotations.json",'r'))
train = []
val = []
train_ann = []
val_ann = []
train_id = []
val_id = []
train_targets = ['/media/ebonetto/WindowsData/pose_zebras_sw/ViTPose/data/real_zebras_old/train/images']
valid_targets = ['/media/ebonetto/WindowsData/pose_zebras_sw/ViTPose/data/real_zebras_old/valid/images']
sf = '/media/ebonetto/WindowsData/real_zebras/ap36k_zebra/valid/images'
for j in f['images']:
    if '13zebra' in j['file_name']:
        tmpfn = 'video'+j['file_name'].split('video')[1].split('_')[0]
        j['file_name'] = 'video'+j['file_name'].split('video')[1].replace('\\','_')
        if tmpfn not in myl:
            train.append(j)
            train_id.append(j['id'])
            for tar in train_targets:
                shutil.copy(f"/media/ebonetto/WindowsData/real_zebras/ap36k_zebra/valid/images/{j['file_name']}", tar)
        else:
            val.append(j)
            val_id.append(j['id'])
            for tar in valid_targets:
                shutil.copy(f"/media/ebonetto/WindowsData/real_zebras/ap36k_zebra/valid/images/{j['file_name']}", tar)
                
for j in f['annotations']:
#     j['category_id'] = 23
#     j['num_keypoints'] = 27
    if j['image_id'] in train_id:
#         j['keypoints'] = map_keypoints(j['keypoints'])
        train_ann.append(j)
    elif j['image_id'] in val_id:
#         j['keypoints'] = map_keypoints(j['keypoints'])
        val_ann.append(j)
    

In [66]:
f2 = json.load(open("/media/ebonetto/WindowsData/pose_zebras_sw/ViTPose/data/apt36k_annotations.json",'r'))

In [67]:
f2['images'] = val
f2['annotations'] = val_ann
# f2['categories'] = [{'name': 'person', 'id': 1, 'supercategory': 'person'},
#  {'supercategory': 'animal',
#   'id': 23,
#   'name': 'zebra',
#   'keypoints': ['left_back_paw',
#    'left_back_knee',
#    'left_back_thigh',
#    'right_back_paw',
#    'right_back_knee',
#    'right_back_thigh',
#    'right_front_paw',
#    'right_front_knee',
#    'right_front_thigh',
#    'left_front_paw',
#    'left_front_knee',
#    'left_front_thigh',
#    'tail_end',
#    'tail_base',
#    'right_ear_tip',
#    'right_ear_base',
#    'left_ear_tip',
#    'left_ear_base',
#    'right_eye',
#    'left_eye',
#    'nose',
#    'neck_start',
#    'neck_end',
#    'skull',
#    'body_middle',
#    'back_end',
#    'back_front'],
#   'skeleton': [[1, 2],
#    [2, 3],
#    [3, 26],
#    [4, 5],
#    [5, 6],
#    [6, 26],
#    [7, 8],
#    [8, 9],
#    [10, 11],
#    [11, 12],
#    [13, 14],
#    [15, 16],
#    [17, 18],
#    [16, 19],
#    [19, 20],
#    [18, 20],
#    [19, 21],
#    [20, 21],
#    [19, 24],
#    [20, 24],
#    [21, 24],
#    [24, 23],
#    [23, 22],
#    [22, 27],
#    [27, 9],
#    [27, 12],
#    [27, 25],
#    [25, 26],
#    [26, 14]]},
#  {'supercategory': 'vehicle', 'id': 5, 'name': 'airplane'}]

In [68]:
json.dump(f2, open("/media/ebonetto/WindowsData/pose_zebras_sw/ViTPose/data/real_zebras_old/valid/valid.json",'w'))

# merge jsons

In [74]:
j1 = json.load(open("/media/ebonetto/WindowsData/pose_zebras_sw/ViTPose/data/zebras_apt_valid.json",'r'))
j2 = json.load(open("/media/ebonetto/WindowsData/pose_zebras_sw/ViTPose/data/zebras/valid/valid.json",'r'))
of = "/media/ebonetto/WindowsData/pose_zebras_sw/ViTPose/data/merged_zebras/valid/valid_full.json"

In [73]:
import random

tmp = {}
tmp['images'] = []
tmp['annotations'] = []
tmp['categories'] = j1['categories']
counter = 1
ann_counter = 1
for j in j1['images']:
    old_id = j['id']
    j['id'] = counter
    tmp['images'].append(j)
    for ann in j1['annotations']:
        if ann['image_id'] == old_id:
            ann['id'] = ann_counter
            ann['image_id'] = counter
            tmp['annotations'].append(ann)
            ann_counter += 1
    counter += 1
for idx, j in enumerate(j2['images']):
    print(f"{idx} over 14401")
#     if random.random() < .9:
#         continue
    old_id = j['id']
    j['id'] = counter
    tmp['images'].append(j)
    for ann in j2['annotations']:
        if ann['image_id'] == old_id:
            ann['id'] = ann_counter
            ann_counter += 1
            ann['image_id'] = counter
            ann['bbox'][0] = max(0,ann['bbox'][0]-10)
            ann['bbox'][1] = max(0,ann['bbox'][1]-10)
            ann['bbox'][2] = min(j['width']-ann['bbox'][0],ann['bbox'][2]+10)
            ann['bbox'][3] = min(j['height']-ann['bbox'][1],ann['bbox'][3]+10)
            tmp['annotations'].append(ann)
    counter += 1
json.dump(tmp, open(of,'w'))

0 over 14401
1 over 14401
2 over 14401
3 over 14401
4 over 14401
5 over 14401
6 over 14401
7 over 14401
8 over 14401
9 over 14401
10 over 14401
11 over 14401
12 over 14401
13 over 14401
14 over 14401
15 over 14401
16 over 14401
17 over 14401
18 over 14401
19 over 14401
20 over 14401
21 over 14401
22 over 14401
23 over 14401
24 over 14401
25 over 14401
26 over 14401
27 over 14401
28 over 14401
29 over 14401
30 over 14401
31 over 14401
32 over 14401
33 over 14401
34 over 14401
35 over 14401
36 over 14401
37 over 14401
38 over 14401
39 over 14401
40 over 14401
41 over 14401
42 over 14401
43 over 14401
44 over 14401
45 over 14401
46 over 14401
47 over 14401
48 over 14401
49 over 14401
50 over 14401
51 over 14401
52 over 14401
53 over 14401
54 over 14401
55 over 14401
56 over 14401
57 over 14401
58 over 14401
59 over 14401
60 over 14401
61 over 14401
62 over 14401
63 over 14401
64 over 14401
65 over 14401
66 over 14401
67 over 14401
68 over 14401
69 over 14401
70 over 14401
71 over 14401
72

561 over 14401
562 over 14401
563 over 14401
564 over 14401
565 over 14401
566 over 14401
567 over 14401
568 over 14401
569 over 14401
570 over 14401
571 over 14401
572 over 14401
573 over 14401
574 over 14401
575 over 14401
576 over 14401
577 over 14401
578 over 14401
579 over 14401
580 over 14401
581 over 14401
582 over 14401
583 over 14401
584 over 14401
585 over 14401
586 over 14401
587 over 14401
588 over 14401
589 over 14401
590 over 14401
591 over 14401
592 over 14401
593 over 14401
594 over 14401
595 over 14401
596 over 14401
597 over 14401
598 over 14401
599 over 14401
600 over 14401
601 over 14401
602 over 14401
603 over 14401
604 over 14401
605 over 14401
606 over 14401
607 over 14401
608 over 14401
609 over 14401
610 over 14401
611 over 14401
612 over 14401
613 over 14401
614 over 14401
615 over 14401
616 over 14401
617 over 14401
618 over 14401
619 over 14401
620 over 14401
621 over 14401
622 over 14401
623 over 14401
624 over 14401
625 over 14401
626 over 14401
627 over 1

1104 over 14401
1105 over 14401
1106 over 14401
1107 over 14401
1108 over 14401
1109 over 14401
1110 over 14401
1111 over 14401
1112 over 14401
1113 over 14401
1114 over 14401
1115 over 14401
1116 over 14401
1117 over 14401
1118 over 14401
1119 over 14401
1120 over 14401
1121 over 14401
1122 over 14401
1123 over 14401
1124 over 14401
1125 over 14401
1126 over 14401
1127 over 14401
1128 over 14401
1129 over 14401
1130 over 14401
1131 over 14401
1132 over 14401
1133 over 14401
1134 over 14401
1135 over 14401
1136 over 14401
1137 over 14401
1138 over 14401
1139 over 14401
1140 over 14401
1141 over 14401
1142 over 14401
1143 over 14401
1144 over 14401
1145 over 14401
1146 over 14401
1147 over 14401
1148 over 14401
1149 over 14401
1150 over 14401
1151 over 14401
1152 over 14401
1153 over 14401
1154 over 14401
1155 over 14401
1156 over 14401
1157 over 14401
1158 over 14401
1159 over 14401
1160 over 14401
1161 over 14401
1162 over 14401
1163 over 14401
1164 over 14401
1165 over 14401
1166 ove

1626 over 14401
1627 over 14401
1628 over 14401
1629 over 14401
1630 over 14401
1631 over 14401
1632 over 14401
1633 over 14401
1634 over 14401
1635 over 14401
1636 over 14401
1637 over 14401
1638 over 14401
1639 over 14401
1640 over 14401
1641 over 14401
1642 over 14401
1643 over 14401
1644 over 14401
1645 over 14401
1646 over 14401
1647 over 14401
1648 over 14401
1649 over 14401
1650 over 14401
1651 over 14401
1652 over 14401
1653 over 14401
1654 over 14401
1655 over 14401
1656 over 14401
1657 over 14401
1658 over 14401
1659 over 14401
1660 over 14401
1661 over 14401
1662 over 14401
1663 over 14401
1664 over 14401
1665 over 14401
1666 over 14401
1667 over 14401
1668 over 14401
1669 over 14401
1670 over 14401
1671 over 14401
1672 over 14401
1673 over 14401
1674 over 14401
1675 over 14401
1676 over 14401
1677 over 14401
1678 over 14401
1679 over 14401
1680 over 14401
1681 over 14401
1682 over 14401
1683 over 14401
1684 over 14401
1685 over 14401
1686 over 14401
1687 over 14401
1688 ove

2153 over 14401
2154 over 14401
2155 over 14401
2156 over 14401
2157 over 14401
2158 over 14401
2159 over 14401
2160 over 14401
2161 over 14401
2162 over 14401
2163 over 14401
2164 over 14401
2165 over 14401
2166 over 14401
2167 over 14401
2168 over 14401
2169 over 14401
2170 over 14401
2171 over 14401
2172 over 14401
2173 over 14401
2174 over 14401
2175 over 14401
2176 over 14401
2177 over 14401
2178 over 14401
2179 over 14401
2180 over 14401
2181 over 14401
2182 over 14401
2183 over 14401
2184 over 14401
2185 over 14401
2186 over 14401
2187 over 14401
2188 over 14401
2189 over 14401
2190 over 14401
2191 over 14401
2192 over 14401
2193 over 14401
2194 over 14401
2195 over 14401
2196 over 14401
2197 over 14401
2198 over 14401
2199 over 14401
2200 over 14401
2201 over 14401
2202 over 14401
2203 over 14401
2204 over 14401
2205 over 14401
2206 over 14401
2207 over 14401
2208 over 14401
2209 over 14401
2210 over 14401
2211 over 14401
2212 over 14401
2213 over 14401
2214 over 14401
2215 ove

2673 over 14401
2674 over 14401
2675 over 14401
2676 over 14401
2677 over 14401
2678 over 14401
2679 over 14401
2680 over 14401
2681 over 14401
2682 over 14401
2683 over 14401
2684 over 14401
2685 over 14401
2686 over 14401
2687 over 14401
2688 over 14401
2689 over 14401
2690 over 14401
2691 over 14401
2692 over 14401
2693 over 14401
2694 over 14401
2695 over 14401
2696 over 14401
2697 over 14401
2698 over 14401
2699 over 14401
2700 over 14401
2701 over 14401
2702 over 14401
2703 over 14401
2704 over 14401
2705 over 14401
2706 over 14401
2707 over 14401
2708 over 14401
2709 over 14401
2710 over 14401
2711 over 14401
2712 over 14401
2713 over 14401
2714 over 14401
2715 over 14401
2716 over 14401
2717 over 14401
2718 over 14401
2719 over 14401
2720 over 14401
2721 over 14401
2722 over 14401
2723 over 14401
2724 over 14401
2725 over 14401
2726 over 14401
2727 over 14401
2728 over 14401
2729 over 14401
2730 over 14401
2731 over 14401
2732 over 14401
2733 over 14401
2734 over 14401
2735 ove

3198 over 14401
3199 over 14401
3200 over 14401
3201 over 14401
3202 over 14401
3203 over 14401
3204 over 14401
3205 over 14401
3206 over 14401
3207 over 14401
3208 over 14401
3209 over 14401
3210 over 14401
3211 over 14401
3212 over 14401
3213 over 14401
3214 over 14401
3215 over 14401
3216 over 14401
3217 over 14401
3218 over 14401
3219 over 14401
3220 over 14401
3221 over 14401
3222 over 14401
3223 over 14401
3224 over 14401
3225 over 14401
3226 over 14401
3227 over 14401
3228 over 14401
3229 over 14401
3230 over 14401
3231 over 14401
3232 over 14401
3233 over 14401
3234 over 14401
3235 over 14401
3236 over 14401
3237 over 14401
3238 over 14401
3239 over 14401
3240 over 14401
3241 over 14401
3242 over 14401
3243 over 14401
3244 over 14401
3245 over 14401
3246 over 14401
3247 over 14401
3248 over 14401
3249 over 14401
3250 over 14401
3251 over 14401
3252 over 14401
3253 over 14401
3254 over 14401
3255 over 14401
3256 over 14401
3257 over 14401
3258 over 14401
3259 over 14401
3260 ove

In [59]:
counter

961

In [57]:
tmp['annotations'][476]

{'bbox': [931.3873901367188,
  199.9016876220703,
  986.1989135742188,
  864.0055389404297],
 'keypoints': [0,
  0,
  0,
  1988.962646484375,
  1128.896240234375,
  2,
  1950.400390625,
  907.1620483398438,
  2,
  0,
  0,
  0,
  1979.322265625,
  1128.896240234375,
  2,
  0,
  0,
  0,
  0,
  0,
  0,
  1304.479248046875,
  1042.130615234375,
  2,
  1381.60400390625,
  907.1620483398438,
  2,
  0,
  0,
  0,
  1381.60400390625,
  1071.052490234375,
  2,
  1429.80712890625,
  897.5214233398438,
  2,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1005.6200561523438,
  405.85003662109375,
  2,
  1005.6200561523438,
  425.13128662109375,
  2,
  1015.2606811523438,
  569.7404174804688,
  2,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'track_id': 1,
 'category_id': 23,
 'image_id': 248,
 'video_id': 260,
 'id': 477,
 'num_keypoints': 27,
 'is_crowd': 0,
 'area': 852081.3238251591}